## Trainning a better model

In [1]:
%matplotlib inline
import utils
import imp
imp.reload(utils)
from utils import *
from __future__ import division, print_function

Using TensorFlow backend.


In [2]:
# path = "data/dogscats/sample/"
path = "data/redux/sample/"
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)

#batch_size = 64
batch_size = 1

#fc_length = 16 * 14 * 14
fc_length = 16 * 16 * 16

## Are we underfitting?

Out validataion accuracy so far has generally been higher than out training accuracy. That leads to two obvious questions:

1. How is this possible?
2. Is this desirable?

The answer to (1) is taht this is happening because of *dropout*. Dropout refers to a layer that randomly deletes (i.e. sets to zero) ach activation in the previous layer with probability *p* (genarally 0.5). This only happens during training, not when calculating the accuracy onthe validation set, which is why the validation set can show higher accuracy than the training set.

The purpose of dropout is to avoid overfitting. By deleting pars of the neural wetowrk at random during training, it ensures that no one part of the network can overfit to one part of the training set. The creation of dropout was one of the key developments in deep learning, and has allowed us to create rich models without overfitting. However, it can also ersult in underfitting if overused, and this is something we should be careful of with our model.

So the answer to (2) is: this is probably not desirable. It is likely that we can get berrter validation set results with less (or no) dropout, if we're seeing that validation accuracy is higher thatn training accuracy - a strong sign of underfitting. So let's try removing dropouts entirely, and see what happens!

(We had dropout in this model alrady because the VGG authors found it necessary for the imagenet competition. But that doesn;t mean it's necessary for fogs vs cats, so we will do our anaaysis of regularization approaches from scratch.)

## Removing dropout

Our high level approach here will be start with our fine-tuned cats vs dogs model (with dropout), then fine-tune all the dense layers, after removing dropout from them. The steps we will take are:

- Re-create and load out modified VGG model with binary dependent (i.e. dogs vs cats)
- SPlit the model between the convolutional (*conv*) layers and the dense layers
- Pre-calculate the output of the conv layers, so that we don't have to redundently re-calculate them on every epoch
- Create a new model with just the dense layers, and dropout p set zero
- Train this new model using the output of the conv layers as training data.

As before we need to start with a working model, so let's bring in our working VGG 16 model and change it to predict our binary dependent...

In [3]:
model = vgg_ft(2)

... and load out fine-tuned weights.

In [4]:
# model.load_weights(model_path + 'finetune3.h5')
model.load_weights(model_path + 'finetune2.h5')

We are going to training a number of iterations without dropout, so it would be best for us to pre-calculate the inut to thte fully connected layers - i.e. the *Flatten()* layer. We'll start by finding this layer in out model, and creating an new model taht contains just the layers up to and including this layer.

In [5]:
layers = model.layers

In [6]:
last_conv_idx = [index for index, layer in enumerate(layers) if type(layer) is Convolution2D][-1]

In [7]:
last_conv_idx

30

In [8]:
layers[last_conv_idx]

In [9]:
conv_layers = layers[:(last_conv_idx + 1)]
conv_model = Sequential(conv_layers)

In [10]:
# Dense layers - also known as fully connected or 'FC' layers
fc_layers = layers[(last_conv_idx + 1):]

Now we can use the exact same approach to creating features as we used when we created the lilnear model from teh imagenet preictions int thelast lesson - it's onlt the model tha thas changed. As you're seeing, there;s a fairly a small number of "recipres' taht can get us a long way!

In [11]:
batches = get_batches(path + 'train', shuffle = False, batch_size = batch_size)
val_batches = get_batches(path + 'valid', shuffle = False, batch_size = batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 200 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [12]:
batches.class_indices

{'cats': 0, 'dogs': 1}

In [13]:
val_features = conv_model.predict_generator(val_batches, val_batches.samples)

In [14]:
trn_features = conv_model.predict_generator(batches, batches.samples)

In [15]:
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [16]:
trn_features = load_array(model_path + 'train_convlayer_features.bc')
val_features = load_array(model_path + 'valid_convlayer_features.bc')

In [17]:
trn_features.shape

(200, 512, 14, 14)

In [18]:
# Copy the weights from the pre-trained model.
# NB: Since we're removing dropout, we wnat to half the weights
def proc_wgts(layer):
    return [o/2 for o in layer.get_weights()]

In [19]:
# Such a finly tuned model needs to be updated very slowly!
opt = RMSprop(lr = 0.00001, rho = 0.7)

In [20]:
def get_fc_model() :
    model = Sequential([
        MaxPooling2D(input_shape = conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(fc_length, activation = 'relu'),
        Dropout(0.),
        Dense(fc_length, activation = 'relu'),
        Dropout(0.),
        Dense(2, activation = 'softmax')
    ])
    
    for l1, l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))
        
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 3, 224, 224)       0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 3, 226, 226)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 224, 224)      1792      
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 64, 226, 226)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 224, 224)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 112, 112)      0         
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 64, 114, 114)      0         
__________

In [22]:
fc_model = get_fc_model()

And fit the model in the usual way:

In [24]:
fc_model.fit(trn_features, trn_labels, epochs = 8, batch_size = batch_size, validation_data = (val_features, val_labels))

C:\Users\hjkim\Anaconda3\lib\site-packages\keras-2.0.8-py3.6.egg\keras\models.py:852: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 200 samples, validate on 50 samples
Epoch 1/8
107/200 [===============>..............] - ETA: 207s - loss: 1.1432 - acc: 0.9065

KeyboardInterrupt: 

In [ ]:
fc_model.save_weights(model_path + 'no_dropout.h5')

In [ ]:
fc_model.load_weights(model_path + 'no_dropout.h5')

### About data augmentation

Keras comes with very convinient features for automating data augmentation. You simply define what types and maximum amounts of agumentation you want, and keras endsures that every item of every batch randomly is changed accordingly to these settings. Here's how to define a generator that includes data augmentation:

In [ ]:
# dim_orderinf = 'tf' uses tensorflow dimmension ordering
# which is the same order as matplotlib uses for display.
# Therefore when hust using for display purposes, this is more convinient
gen = image.ImageDataGenerator(rotation_range = 10, width_shift_range = 0.1, height_shift_range = 0.1, shear_range = 0.15, zoom_range = 0.1, channel_shift_range = 10, horizontal_flip = True, dim_ordering = 'tf')

Let's take a look at how this generator changes a single image (the details of this code don't matter much, but feel free to read the comments and keras docs to understand the details if you're interested).

In [ ]:
# Create a 'batch' of a single image
img = np.expand_dims(ndimage,imread('data/dogscats/test/7.jpg'), 0)
# request the generator to create batches from this iamge
aug_imgs = [next(aug_iter)[0].astype(np.int8) for in in range(8)]

In [ ]:
# The original
plt.imshow(img[0])

In [ ]:
# Augmented_data
plots(aug_imgs, (20, 7), 2)

In [ ]:
# Ensure taht we return to theano dimension ordering
K.set_image_dim_ordering('th')

### Adding data augmentation

Let's try adding a small amount of data augmentation, andn see if we reduce overfitting as a result. The approach will be identicalt o the method we used to finetune the dense layeys in lesson 2, except tha twe will use a generator with augmentation configured. Here's how we setup the generator, and create batches from it:

In [ ]:
gen = image.ImageDataGenerator(rotation_range = 15, width_shift_range = 0.1, height_shift_range = 0.1, zoom_range = 0.1, horizontal_flip = True)

In [ ]:
batches = get_batches(path + 'train', gen, batch_size = batch_size)
# NBL We don't want to augment or shuffle the validation set
val_batches = get_batches(path = 'valid', shuffle= = False, batch_size = batch_size)

When using data augmentation, we can't pre-compute out convolutional layer features, since randomized changes are being made to every input image. That is, even if the training process sees the same image multiple time, each time it will have undergone different data augmentation, so the results of the convolutional layers will be different.

Therefore, in order to alow data to flow through all the conv layers and our new dense layers, we attach out fully connected model to the convolutional model -- after ensuring that the convolutional layers are not trainable:

In [ ]:
fc_model = get_fc_model()

In [ ]:
for layer in conv_model.layers: layer.trainable = False
# Look how easy it is to connect two models together!
conv_model.add(fc_model)

Now we can compile, train, and save our models as usual - note that we use *fit_generator()* since we want to pull random images from the directories on every batch.

In [ ]:
conv_model.compile(optimizer =opt, loss = 'categorical_cross_entorpy', metrics = ['accuracy'])

In [ ]:
conv_model.fit_generator(batche, samples_per_epoch = batches.nb_sample, epochs = 8, validation_data = val_batches, nb_val_samples = val_batches.samples)

In [ ]:
conv_model.fit_generator(batche, samples_per_epoch = batches.nb_sample, epochs = 3, validation_data = val_batches, nb_val_samples = val_batches.samples)

In [ ]:
conv_model.save_weigths(model_path + 'aug1.h5')
conv_model.load_weigths(model_path + 'aug1.h5')

## Batch normalization

### About batch normalization

Batch normalization (*batchnorm*) is a way to ensure that activations don;t become too high or too low at any point in the model. Adjusting activations so they are similar scales is called *normalization*. Normalization is very helpful for fast training - if some activations are bery high, they will saturate the model and create very large gradients, causing training to fail; if very low, they will cause training to proceed very slowly. Furthermore, large or small activations in one layer will tend to result in even larger or smaller activations in later layers, since the activations het multiplied repeatedly across th e layers.

Prior to the development of abtnorm in 2015, only yhe inputs to a model could be effectively normalized - by simply subtracting their mean and dividing by their standard deviation. However, wieghts in intermediate layers could easily become poorly scaled, due to problems in weight initialization, or a high learning rate combined with random fluctuations in wieghts.

Batchnorm resolves this problem by normalizing each intermediate layers as well. The details of how it works are not terribly important (although Iwll outline them in a monment) - the important takaway is that **all modern networks should use batchnorm, or something equivalent**. There are two erasons for this:

1. Adding batchnorm to ao model can result in **10x or more improvements in training speed**
2. Because normalization greatly reduces the ability of a small number of outlying inputs to over-influence the training, it also tends to **reduce overfitting**.

As promised, here's a brief outlien of how batchnorm works. As a first step, it normalizes intermediate layers in the same way as inpu t layers can be normalized. But this on its own would not be enough, since the model would then just push the weights up or down indefinitely to try to unde this normalization. Therefore, batchnorm takes two additional steps:

1. Add two more trainable parameters to each layers - on to multiply all activations to set an arbitary standard deviation, and ont o add to all activations to set an arbitary mean
2. Incorporate both the normalization, and the learnt multiply/add parameters, into the gradient calculations during backprop.

This ensures that the weights don't tend to push very high or very low (since the normalization is included in the gradient calculations, so the updates are aware of the normalization). But it also ensures that if a layer does need to change the overall mean or standard deciation in roder to match the output scale, it can do so.

### Adding batchnorm to the model

We can use nearly the same approach as before - but this time we'll add batchnorm layers (adn dropout layers):

In [ ]:
conv_layers[-1].output_shape[1:]

In [ ]:
def get_bn_layers(p) :
    return [
        MaxPooling2D(input_shape = conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(fc_length, activation = 'relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(fc_length, activation = 'relu'),
        BatchNormalization(),
        Dropout(p),
        Dense(1000, activation = 'softmax')
    ]

In [ ]:
def load_fc_weights_from_vgg16bn(model):
    'Load weights for model from the dense layers of the Vgg16BN model'
    # See imagenet_batchnorm.ipynb for info on how the iehghts for
    # vgg16bn can be generated from the standard vgg16 weights.
    from vgg16bn import Vgg16BN
    vgg16_bn = Vgg16BN
    _, fc_layers = split_at(vgg16_bn.model, Convolution2D)
    copy_weights(fc_layers, model.layers)


In [ ]:
p = 0.6

In [ ]:
bn_mode = Sequential(get_bn_layers(0.6))

In [ ]:
load_fc_weights_from_vgg16bn(bn_model)

In [ ]:
def proc_wgts(layer, prev_p, new_p) :
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

In [ ]:
for l in bn_model.layers:
    if type(l) == Dense: l.set_weights(proc_wgts(l, 0.5, 0.6))

In [ ]:
bn_model.pop()
for layer in bn_model.layers: layer.trainable = False

In [ ]:
bn_model.add(Dense(2, activation = 'softmax'))

In [ ]:
bn_model.compile(Adam(), 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
bn_model.fit(trn_features, trn_labels, epochs = 8, validation_data = (val_features, val_labels))

In [ ]:
bn_model.save_weights(model_path + 'bn.h5')

In [ ]:
bn_model.load_weights(model_path + 'bn.h5')

In [ ]:
bn_layers = get_bn_layers(0.6)
bn_layers.pop()
bn_layers.append(Dense(2, activation = 'softmax'))

In [ ]:
final_model = Sequential(conv_layers)
for layer in final_model.layers: layer.trainable = False
for layer in bn_layers: final_mode.add(layer)

In [ ]:
for l1, l2 in zip(bn_model.layers, bn_layers):
    l2.set_weights(l1.get_weights())

In [ ]:
final_model.compile(optimizer = Adam(), loss = 'categorical_crossentorpy', metrics = ['accuracy'])

In [ ]:
final_model.fit_generator(batches, samples_per_epoch = batches.samples, nb_epoch = 1, validation_data = val_batches, nb_val_samples = val_batches.samples)

In [ ]:
final_model.save_weights(model_path + 'final1.h5')

In [ ]:
final_model.fit_generator(batches, samples_per_epoch = batches.samples, nb_epoch = 4, validation_data = val_batches, nb_val_samples = val_batches.samples)

In [ ]:
final_model.save_weights(model_path + 'final2.h5')

In [ ]:
final_model.optimizer.lr = 0.001

In [ ]:
final_model.fit_generator(batches, smaples_per_epoch = batches.samples, nb_epoch = 4, validation_dat = val_batches, nb_val_samples = val_batches.samples)

In [ ]:
bn_model.save_weights(model_path + 'final3.h5')